Write as a script taking the review_id as input, or a set of functions

In [ ]:
import ast
import openai
import os

from dotenv import load_dotenv
from openai import OpenAI
from src.utils.db import PostgreSQLDatabase

In [ ]:
review_id = 1013303

# Connect to the db

In [ ]:
db = PostgreSQLDatabase()
db.connect()

# Get review and build prompt

In [ ]:
review = db.query_data('reviews_raw', condition=f'review_id = {review_id}')

text = review[0][3] + f"\n\n" + review[0][4]

prompt = f"""
Instructions:
Below is a movie review that I want you to analyze. 
For each of the following aspect, you must determine if it is mentioned in the review, and if it is, what is the corresponding sentiment on the following scale: *very negative*, *negative*, *neutral* (including mixed or contradictory sentiments), *positive*, *very positive*.
- *Storytelling* (including characters and their development, narrative progression, plot twists, screenplay, dialogues, overall pacing)
- *Acting performance* (including vocal, musical, danse, or stunt work if applicable)
- *Cinematography and visual style* (including colors and lightening, set design, costumes, makeup, special effects, overall aesthetic of the film)
- *Music and sound design* (including soundtrack and scores)
- *Theme and values* (including the moral or political message, emotional resonance, cultural or societal impact)
Your answer should take the form of a Python list with each aspect, whether it is mentioned, and the corresponding sentiment (which should be 'NA' if the aspect is not mentioned).
Additionally, I want you to identify the overall sentiment about the movie that the review conveys: is the movie *excellent*, *good despite minor flaws*, *average*, *bad despite some qualities*, *terrible*. This overall sentiment should be appended to the previous list.
You must return this list only, without any additional commentary. Your answer should have the following format (with example values):
 [('Storytelling', 'not mentioned', 'NA'), ('Acting performance', 'mentioned', 'good'), ('Cinematography and visual style', 'mentioned', 'very negative'), ('Music and sound design', 'mentioned', 'very good'), ('Theme and values', 'not mentioned', 'NA'), ('Overall', 'average')]

Now the review:
{text}
"""

# Send prompt

In [ ]:
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
client = OpenAI()

def get_GPT_answer(prompt):    
    try:
        completion = client.chat.completions.create(
            model = "gpt-4o-mini",
            messages = [{"role": "user", "content": prompt}]
        )
        return "request worked", completion

    except Exception as e:
        return "request failed", str(e)

In [ ]:
answer = get_GPT_answer(prompt)

In [ ]:
results = ast.literal_eval(answer[1].choices[0].message.content)

In [ ]:
results

# Write to DB

In [ ]:
# Sentiment to integer mapping
sentiment_mapping = {
    'very negative': -2,
    'negative': -1,
    'neutral': 0,
    'positive': 1,
    'very positive': 2,
    'awful': -2,
    'bad despite some qualities': -1,
    'average': 0,
    'good despite minor flaws': 1,
    'excellent': 2,
    'NA': None
}

mapped_values = [None] * 6

# Map aspect-based sentiments
for i, (label, _, sentiment) in enumerate(results[:5]):
    if sentiment in sentiment_mapping:
        mapped_values[i] = sentiment_mapping[sentiment]

# Map overall sentiment
label, sentiment = results[5]
if sentiment in sentiment_mapping:
    mapped_values[5] = sentiment_mapping[sentiment]

data = [(review_id, *mapped_values)]

In [ ]:
db.insert_data('reviews_sentiments', data)

In [ ]:
print("\nAll reviews:")
reviews = db.query_data('reviews_sentiments')
for review in reviews:
    print(review)

In [ ]:
db.close_connection()